You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

In [169]:
import pandas as pd

In [170]:
Office_addresses = pd.read_csv("datasets/office_addresses.csv")
Office_addresses.shape

(3, 5)

In [171]:
Office_addresses.head()

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [172]:
import json

with open('datasets/employee_roles.json') as f:
   data = json.load(f)

print(data)

{'A2R5H9': {'title': 'CEO', 'monthly_salary': '$4500', 'team': 'Leadership'}, 'H8K0L6': {'title': 'CFO', 'monthly_salary': '$4500', 'team': 'Leadership'}, 'G4R7V0': {'title': 'Business Developer', 'monthly_salary': '$3000', 'team': 'Sales'}, 'M1Z7U9': {'title': 'Office Manager', 'monthly_salary': '$2000', 'team': 'People Operations'}}


In [173]:
Employee_roles = pd.read_json("datasets/employee_roles.json")
Employee_roles.shape

(3, 4)

In [174]:
Employee_roles.head()

,A2R5H9,H8K0L6,G4R7V0,M1Z7U9
title,CEO,CFO,Business Developer,Office Manager
monthly_salary,$4500,$4500,$3000,$2000
team,Leadership,Leadership,Sales,People Operations


In [175]:
employee_addresses = pd.read_excel('datasets/employee_information.xlsx', sheet_name = 'employee_addresses')
employee_addresses.shape

(4, 7)

In [176]:
employee_addresses.head()

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [177]:
col_names = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship']

In [178]:
emergency_contacts = pd.read_excel('datasets/employee_information.xlsx', sheet_name = 'emergency_contacts',header=None, names=col_names)
emergency_contacts.shape

(4, 6)

In [179]:
emergency_contacts.head()

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [180]:
Employee_roles = Employee_roles.T 
Employee_roles

,title,monthly_salary,team
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


In [181]:
Employee_roles.index.names = ['employee_id']
Employee_roles

,title,monthly_salary,team
employee_id,,,
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


In [182]:
employees_final = Employee_roles.merge(employee_addresses, on='employee_id', how='left')
employees_final

,employee_id,title,monthly_salary,team,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,CEO,$4500,Leadership,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,CFO,$4500,Leadership,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Business Developer,$3000,Sales,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Office Manager,$2000,People Operations,Coates,Tig,FR,Paris,Rue de l'Université,7


In [183]:
employees_final.rename(columns={'employee_first_name':'first_name', 'employee_last_name':'last_name', '':''}, inplace=True)
employees_final

,employee_id,title,monthly_salary,team,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,CEO,$4500,Leadership,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,CFO,$4500,Leadership,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Business Developer,$3000,Sales,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Office Manager,$2000,People Operations,Coates,Tig,FR,Paris,Rue de l'Université,7


In [184]:
emergency_contacts = emergency_contacts.filter(['employee_id', 'emergency_contact', 'emergency_contact_number', 'relationship'])
emergency_contacts

,employee_id,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Venus Noone,+1-202-555-0130,Wife


In [185]:
employees_final = employees_final.merge(emergency_contacts, on='employee_id', how='left')
employees_final

,employee_id,title,monthly_salary,team,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,CEO,$4500,Leadership,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,CFO,$4500,Leadership,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Business Developer,$3000,Sales,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Office Manager,$2000,People Operations,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife


In [186]:
Office_addresses['employee_city'] = Office_addresses['office_city'].replace(['New York City'], 'New-York')
Office_addresses

,office,office_country,office_city,office_street,office_street_number,employee_city
0,Leuven Office,BE,Leuven,Martelarenlaan,38,Leuven
1,ESB Office,US,New York City,Fifth Avenue,350,New-York
2,WeWork Office,GB,London,Old Street,207,London


In [187]:
employees_final = employees_final.merge(Office_addresses, on='employee_city', how='left')
employees_final

,employee_id,title,monthly_salary,team,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,CEO,$4500,Leadership,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,CFO,$4500,Leadership,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Business Developer,$3000,Sales,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Office Manager,$2000,People Operations,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,NaN,NaN,NaN,NaN,NaN


In [188]:
employees_final[['office', 'office_country', 'office_city', 'office_street', 'office_street_number']] = employees_final[['office', 'office_country', 'office_city', 'office_street', 'office_street_number']].fillna("Remote")
employees_final

,employee_id,title,monthly_salary,team,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,CEO,$4500,Leadership,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,CFO,$4500,Leadership,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Business Developer,$3000,Sales,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Office Manager,$2000,People Operations,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Remote,Remote,Remote,Remote,Remote


In [189]:
employees_final = employees_final.set_index('employee_id')
employees_final

,title,monthly_salary,team,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,CEO,$4500,Leadership,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,CFO,$4500,Leadership,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Business Developer,$3000,Sales,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Office Manager,$2000,People Operations,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Remote,Remote,Remote,Remote,Remote


In [190]:
employees_final = employees_final.filter(['first_name', 'last_name', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number'])

In [191]:
employees_final

,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
